In [14]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from transformers import AutoModel, AutoProcessor
import warnings
warnings.filterwarnings('ignore')

model = AutoModel.from_pretrained('Marqo/marqo-fashionCLIP', trust_remote_code=True)
processor = AutoProcessor.from_pretrained('Marqo/marqo-fashionCLIP', trust_remote_code=True)

You are using a model of type clip to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [15]:
df = pd.read_csv('myntradataset/styles.csv', usecols=['id',  'subCategory', 'articleType'])
print(f"Style {type(df)} {df.shape}")
print(df.head(10))

unique, counts = np.unique(df["articleType"].tolist(), return_counts=True)
print(f"{unique.shape[0]} Classes:\n{unique}\n{counts}")

Style <class 'pandas.core.frame.DataFrame'> (44446, 3)
      id subCategory  articleType
0  15970     Topwear       Shirts
1  39386  Bottomwear        Jeans
2  59263     Watches      Watches
3  21379  Bottomwear  Track Pants
4  53759     Topwear      Tshirts
5   1855     Topwear      Tshirts
6  30805     Topwear       Shirts
7  26960     Topwear       Shirts
8  29114       Socks        Socks
9  30039     Watches      Watches
143 Classes:
['Accessory Gift Set' 'Baby Dolls' 'Backpacks' 'Bangle' 'Basketballs'
 'Bath Robe' 'Beauty Accessory' 'Belts' 'Blazers' 'Body Lotion'
 'Body Wash and Scrub' 'Booties' 'Boxers' 'Bra' 'Bracelet' 'Briefs'
 'Camisoles' 'Capris' 'Caps' 'Casual Shoes' 'Churidar' 'Clothing Set'
 'Clutches' 'Compact' 'Concealer' 'Cufflinks' 'Cushion Covers' 'Deodorant'
 'Dresses' 'Duffel Bag' 'Dupatta' 'Earrings' 'Eye Cream' 'Eyeshadow'
 'Face Moisturisers' 'Face Scrub and Exfoliator' 'Face Serum and Gel'
 'Face Wash and Cleanser' 'Flats' 'Flip Flops' 'Footballs' 'Formal Shoes

In [21]:
# image = [Image.open("outputs/t-shirt.jpeg")]
image = [Image.open("outputs/flipflops.jpg")]
# text = ["hat", "t-shirt", "shoes", 'gloves', 'headwear']
text = list(set(
    list(df["articleType"].unique()) 
    + list(df["subCategory"].unique())
))

processed = processor(text=text, images=image, padding='max_length', return_tensors="pt")

with torch.no_grad():
    image_features = model.get_image_features(processed['pixel_values'], normalize=True)
    text_features = model.get_text_features(processed['input_ids'], normalize=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)
# [0.99990773, 0.00006382, 0.00002847]
top_result = np.argmax(text_probs).item()
print(top_result, text[top_result])

Label probs: tensor([[1.7655e-08, 1.5026e-09, 2.6647e-08, 7.1111e-12, 1.5650e-10, 1.8047e-09,
         2.9565e-05, 4.1409e-09, 5.1403e-12, 5.1202e-09, 1.9282e-12, 8.8848e-09,
         3.2063e-13, 8.7564e-11, 1.3774e-10, 8.9143e-01, 6.5542e-12, 3.9357e-10,
         7.4156e-12, 3.5896e-08, 8.2258e-09, 6.9122e-11, 3.1701e-09, 2.7519e-07,
         2.7358e-07, 1.0848e-09, 3.1018e-10, 1.4207e-10, 7.2185e-12, 1.5128e-10,
         1.3050e-11, 5.0978e-07, 3.2331e-09, 7.2880e-06, 2.3863e-07, 4.6018e-08,
         3.4164e-11, 1.1347e-08, 1.5727e-11, 2.3805e-11, 9.5078e-09, 2.4650e-10,
         8.2877e-12, 5.2105e-10, 4.9835e-10, 3.0663e-10, 1.5115e-10, 5.0888e-08,
         2.9796e-08, 2.0812e-09, 1.8221e-09, 6.2800e-10, 1.3150e-11, 1.0802e-08,
         3.7689e-07, 3.0336e-08, 9.9666e-11, 6.3632e-02, 3.2158e-08, 5.2163e-11,
         2.8752e-07, 2.9698e-12, 1.0200e-09, 6.8365e-10, 2.4579e-09, 2.7840e-10,
         2.9283e-13, 4.2408e-09, 1.9445e-10, 1.2660e-06, 2.0053e-10, 1.3030e-07,
         1.9884

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os

HOME: str = os.getenv('HOME') # echo $HOME
USER: str = os.getenv('USER') # echo $USER
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_models = [
	"openai/clip-vit-base-patch32", # original
	"openai/clip-vit-large-patch14",
	"laion/CLIP-ViT-H-14-laion2B-s32B-b79K",
]

if USER=="farid": # local laptop
	WDIR = os.path.join(HOME, "datasets")
	models_dir = os.path.join(WDIR, "trash", "models")
	model_fpth = pretrained_models[0]
elif USER=="alijanif": # Puhti
	WDIR = "/scratch/project_2004072/ImACCESS"
	models_dir = os.path.join(WDIR, "trash", "models")
	model_fpth = pretrained_models[1]
else: # Pouta
	WDIR = "/media/volume/ImACCESS"
	models_dir = os.path.join(HOME, WDIR, "models")
	model_fpth = pretrained_models[1]

# Load the CLIP model and processor
model = CLIPModel.from_pretrained(model_fpth, cache_dir=models_dir)
processor = CLIPProcessor.from_pretrained(model_fpth, cache_dir=models_dir)

# Load and preprocess images
# "https://www.archives.gov/files/research/military/ww2/photos/images/ww2-14.jpg"
base_url = "https://www.archives.gov/files/research/military/ww2/photos/images/ww2-"
image_urls = [f"{base_url}{i:02d}.jpg" for i in range(1, 203)]
print(f"len(image_urls): {len(image_urls)}")

images = []
for url in image_urls:
  response = requests.get(url)
  img = Image.open(BytesIO(response.content))
  images.append(img)

In [ ]:
# Encode images and text
text = "Cemetery"
inputs = processor(text=[text], images=images, return_tensors="pt", padding=True)

# Compute similarity scores
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
print(logits_per_image.shape, type(logits_per_image))
probs = logits_per_image.softmax(dim=1)  # Convert logits to probabilities
print(probs.shape, type(probs))


In [ ]:
# Ensure top_k is not greater than the number of images
top_k = min(3, len(images))
top_k_indices = probs.topk(top_k, dim=0).indices.squeeze().tolist()
print(top_k_indices, type(top_k_indices))
# Handle the case where top_k_indices is not a list (e.g., when there's only one image)
if isinstance(top_k_indices, int):
  top_k_indices = [top_k_indices]

top_k_images = [images[i] for i in top_k_indices]
print(top_k_images) # [<PIL.JpegImagePlugin.JpegImageFile image mode=L size=1100x1396 at 0x78A40EF4B910>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=1104x1387 at 0x78A40EF4BBB0>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=1437x1066 at 0x78A40EA22730>]

In [ ]:
fig, axes = plt.subplots(1, top_k, figsize=(15, 5))
for i, img in enumerate(top_k_images):
    axes[i].imshow(img)
    axes[i].axis('off')
plt.show()